### MNIST 데이터 불러오기
0~9 숫자 이미지 데이터셋

In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, as_frame= False) # as_frame 넣어줘야 함
X, y = mnist["data"], mnist["target"]
print(X.shape, y.shape)

(70000, 784) (70000,)


위 코드로 불러올 수 있지만 ... 너무 오래 걸리는 것 같아서 구글링 한대로 데이터 셋 다운받는 게 낫나 ㅠㅠ 

In [2]:
# 데이터 가져오는 방법 차이로 인해 아래 과정 필요함
# https://somjang.tistory.com/entry/%ED%95%B8%EC%A6%88%EC%98%A8%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-127%ED%8E%98%EC%9D%B4%EC%A7%80-MNIST-%EC%BD%94%EB%93%9C-ValueError-The-number-of-classes-has-to-be-greater-than-one-got-1-class-%ED%95%B4%EA%B2%B0%EB%B0%A9%EB%B2%95
import numpy as np
y = y.astype(np.int8)

### DATA SET 분리 ( HOLD OUT )

In [3]:
# 오픈 데이터이기 때문에 결측치 등은 존재 X, 랜덤으로 존재하기 때문에 그냥 슬라이싱으로도 가능 ! 
# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=target, random_state=34)
x_train, x_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
print(x_train.shape,x_test.shape)

(60000, 784) (10000, 784)


## I. 이진 분류
두 개의 클래스를 구분하는 분류 방식 → YES or NO !

In [4]:
y_train_5 = (y_train == 5) # 5일 때만 True 
y_test_5 = (y_test == 5)

### SGD 분류기로 학습하기
-> 선형 분류기의 한 종류로, 확률적경사하강법을 이용함

In [6]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(x_train, y_train_5)

SGDClassifier(random_state=42)

In [7]:
some_digit = X[0] # 5를 나타내는 784개의 픽셀 값 ( x )
print(sgd_clf.predict([some_digit])) # 학습 결과로 예측된 y값

[ True]


### K-Cross Validation

In [8]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(sgd_clf, x_train, y_train_5, cv=3, scoring="accuracy"))

[0.95035 0.96035 0.9604 ]


### Confusion Matrix ( 오차행렬 )

In [11]:
from sklearn.model_selection import cross_val_predict
y_train_pred = cross_val_predict(sgd_clf, x_train, y_train_5, cv=3)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_train_5, y_train_pred) # 원본과 예측 값 전달
print(cm)

[[53892   687]
 [ 1891  3530]]


## 다중 분류 : SVCM으로 구현하기 

In [15]:
from sklearn.svm import SVC
svm_clf = SVC()
svm_clf.fit(x_train, y_train)
print(svm_clf.predict([some_digit])) #5

[5]


In [16]:
print(svm_clf.decision_function([some_digit]))

[[ 1.72501977  2.72809088  7.2510018   8.3076379  -0.31087254  9.3132482
   1.70975103  2.76765202  6.23049537  4.84771048]]


In [24]:
# 에러 분석을 위한 confusion matrix
y_train_pred = cross_val_predict(svm_clf, x_train, y_train, cv=3)
conf_mx = confusion_matrix(y_train, y_train_pred)
print(conf_mx)

[[5863    3   10    2    4    8   14    2   13    4]
 [   1 6664   26    9   10    1    3   10   10    8]
 [  19    9 5811   16   26    5    8   30   28    6]
 [   3    9   54 5891    2   62    1   30   61   18]
 [   5   11   13    0 5712    0   20    9    4   68]
 [  11    6   10   57   14 5255   35    4   17   12]
 [  20    8    7    0   10   32 5831    0   10    0]
 [   5   21   42    6   35    2    1 6100    7   46]
 [   8   29   19   38   20   39   21    9 5646   22]
 [  14   11    4   31   72   16    2   59   19 5721]]


## 다중 레이블 분류 : KNN

In [26]:
from sklearn.neighbors import KNeighborsClassifier

# 타겟 벡터 변경
# 벡터 = 시리즈
y_train_large = (y_train >= 7) # 7 이상이면 True 
y_train_odd = (y_train%2 == 1) # 홀수이면 True

# c_는 column형으로 생성 
y_multilabel = np.c_[y_train_large, y_train_odd]

knn_clf = KNeighborsClassifier()
knn_clf.fit(x_train, y_multilabel)

knn_clf.predict([some_digit]) # 7 이상 False, 홀수 True

array([[False,  True]])